In [75]:
import streamlit as st   
import pandas as pd 
import plotly.express as px 


In [76]:
df = pd.read_csv('../vehicles_us.csv')

In [77]:
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

In [78]:
df.shape, df.dtypes, df.head()

((51525, 13),
 price             int64
 model_year      float64
 model            object
 condition        object
 cylinders       float64
 fuel             object
 odometer        float64
 transmission     object
 type             object
 paint_color      object
 is_4wd          float64
 date_posted      object
 days_listed       int64
 dtype: object,
    price  model_year           model  condition  cylinders fuel  odometer  \
 0   9400      2011.0          bmw x5       good        6.0  gas  145000.0   
 1  25500         NaN      ford f-150       good        6.0  gas   88705.0   
 2   5500      2013.0  hyundai sonata   like new        4.0  gas  110000.0   
 3   1500      2003.0      ford f-150       fair        8.0  gas       NaN   
 4  14900      2017.0    chrysler 200  excellent        4.0  gas   80903.0   
 
   transmission    type paint_color  is_4wd date_posted  days_listed  
 0    automatic     SUV         NaN     1.0   6/23/2018           19  
 1    automatic  pickup       whi

In [79]:
df.isna().mean().sort_values(ascending=False).head(15)

is_4wd          0.503697
paint_color     0.179854
odometer        0.153168
cylinders       0.102086
model_year      0.070238
condition       0.000000
model           0.000000
price           0.000000
fuel            0.000000
type            0.000000
transmission    0.000000
date_posted     0.000000
days_listed     0.000000
dtype: float64

In [80]:
df.duplicated().sum()

np.int64(0)

In [81]:
df.describe(include="all").T.head(20)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
price,51525.0,NaN,NaN,NaN,12132.46492,10040.803015,1.0,5000.0,9000.0,16839.0,375000.0
model_year,47906.0,NaN,NaN,NaN,2009.75047,6.282065,1908.0,2006.0,2011.0,2014.0,2019.0
model,51525,100,ford f-150,2796,NaN,NaN,NaN,NaN,NaN,NaN,NaN
condition,51525,6,excellent,24773,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cylinders,46265.0,NaN,NaN,NaN,6.125235,1.66036,3.0,4.0,6.0,8.0,12.0
fuel,51525,5,gas,47288,NaN,NaN,NaN,NaN,NaN,NaN,NaN
odometer,43633.0,NaN,NaN,NaN,115553.461738,65094.611341,0.0,70000.0,113000.0,155000.0,990000.0
transmission,51525,3,automatic,46902,NaN,NaN,NaN,NaN,NaN,NaN,NaN
type,51525,13,SUV,12405,NaN,NaN,NaN,NaN,NaN,NaN,NaN
paint_color,42258,12,white,10029,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
num_cols = df.select_dtypes("number").columns.tolist()
cat_cols = df.select_dtypes("object").columns.tolist()
num_cols, cat_cols

(['price', 'model_year', 'cylinders', 'odometer', 'is_4wd', 'days_listed'],
 ['model',
  'condition',
  'fuel',
  'transmission',
  'type',
  'paint_color',
  'date_posted'])

In [83]:
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(page_title="Vehicle Explorer", layout="wide")

st.header("Vehicle Explorer")

# --- Load data ---
# Put the CSV in the same folder as app.py, or change the path here.
DATA_PATH = "vehicles_us.csv"
@st.cache_data
def load_data(path):
    return pd.read_csv(path)

try:
    df = load_data(DATA_PATH)
except FileNotFoundError:
    st.error(f"Could not find {DATA_PATH}. Place it next to app.py or fix DATA_PATH.")
    st.stop()

st.write(f"Loaded **{len(df):,}** rows.")

# --- Optional cleanup helpers (adapt to your columns) ---
# Normalize column names for convenience
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# Quick peek
with st.expander("Preview data"):
    st.dataframe(df.head(20), use_container_width=True)

# --- Checkbox (changes behavior of plots) ---
max_price = st.number_input("Max price filter", min_value=0, value=50_000, step=1000)
only_under_max = st.checkbox(f"Show only cars priced ≤ {max_price:,}", value=False)

df_show = df.copy()
if only_under_max and "price" in df_show.columns:
    df_show = df_show[df_show["price"] <= max_price]

# --- Histogram ---
if "price" in df_show.columns:
    st.subheader("Price distribution")
    fig_hist = px.histogram(df_show, x="price", nbins=50, title="Price Distribution")
    st.plotly_chart(fig_hist, use_container_width=True)
else:
    st.info("Add a column named 'price' to see the price histogram.")

# --- Scatter ---
# Try a few common column names if your CSV differs
possible_x = ["odometer", "mileage", "odometer_value"]
x_col = next((c for c in possible_x if c in df_show.columns), None)
y_col = "price" if "price" in df_show.columns else None

if x_col and y_col:
    st.subheader(f"{y_col.title()} vs {x_col.title()}")
    fig_scatter = px.scatter(
        df_show,
        x=x_col, y=y_col,
        opacity=0.5,
        title=f"{y_col.title()} vs {x_col.title()}"
    )
    st.plotly_chart(fig_scatter, use_container_width=True)
else:
    st.info("Need columns for both 'price' and an odometer-like column (e.g., 'odometer').")


2025-09-30 23:16:45.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-30 23:16:45.582 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-30 23:16:45.582 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-30 23:16:45.583 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-30 23:16:45.584 No runtime found, using MemoryCacheStorageManager
2025-09-30 23:16:45.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-30 23:16:45.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-30 23:16:45.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-30 23:16:45.588 Thread 'MainThread':